In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam,RMSprop,Adadelta,Adagrad,Adamax,Nadam
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/m.tech project solar/weather_2017+2018_hourly.csv")
data_as_np = dataset.GHI.values

In [ ]:
def series_to_supervised(data_np,n_lags,n_outs):
    samples = len(data_np)-(n_lags+n_outs-1)
    Inputs = []
    Targets = np.empty((samples,n_outs))
    for i in range(samples):
        step = i+n_lags
        sample = data_np[i:step]
        Inputs.append(sample)
        label = data_np[step:(step+n_outs)]
        j = 0
        for a in label:
            Targets[i,j] = a
            j = j+1
    return np.array(Inputs),Targets


In [ ]:
def plot_model_history(model_summary):
    plt.plot(model_summary.history['loss'])
    plt.plot(model_summary.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'],loc='upper right')
    plt.show()

Optimizers...

class *Adadelta*: Optimizer that implements the Adadelta algorithm.

class *Adagrad*: Optimizer that implements the Adagrad algorithm.

class *Adam*: Optimizer that implements the Adam algorithm.

class *Adamax*: Optimizer that implements the Adamax algorithm.

class *Ftrl*: Optimizer that implements the FTRL algorithm.

class *Nadam*: Optimizer that implements the Nadam algorithm.

class *Optimizer*: Abstract optimizer base class.

class *RMSprop*: Optimizer that implements the RMSprop algorithm.

class *SGD*: Gradient descent (with momentum) optimizer.

Activation functions.. [link text](https://keras.io/api/layers/activations/)


In [ ]:
# fit a model
def create_model(n_input,n_out, train_x,train_y,test_x,test_y,rate):

    # define model
    model = Sequential()
    model.add(Dense(100, activation='relu',  input_dim=n_input))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_out, activation='relu'))

    cp1 = ModelCheckpoint('MLPs/', save_best_only = True)
    model.compile(loss='mse', optimizer=  RMSprop(learning_rate=rate), metrics=[RootMeanSquaredError()])
    model_summary = model.fit(train_x, train_y, validation_data = (test_x,test_y), epochs=80, batch_size=64, callbacks=[cp1])
    plot_model_history(model_summary)

    model = load_model('MLPs/')

    return model



In [ ]:
data_np = data_as_np.reshape(-1,1)
n_lags = 480
n_out = 24

# Prepare data
Inputs,Targets = series_to_supervised(data_np,n_lags,n_out)

Inputs_model, Targets_model = Inputs[:-10], Targets[:-10]
Inputs_plot, Targets_plot = Inputs[-10:], Targets[-10:]

no_samples = Targets_model.shape[0]
split = int(0.3 * no_samples)

features = 1
n_input = n_lags * features

# Splitting data into train and test data set
Inputs_model = Inputs_model.reshape((no_samples,n_input))

train_x,train_y = Inputs_model[:-split],Targets_model[:-split]
test_x,test_y = Inputs_model[-split:],Targets_model[-split:]



In [ ]:
def rmse_error(pred,y):
  return np.sqrt(mean_squared_error(pred,y))

In [ ]:
learning_rate = [pow(10,x) for x in range(-2,-6,-1)]
estimators = 30
rmse = []
for rate in learning_rate:
  train_rmse_error, test_rmse_error = 0, 0
  for iter in range(estimators):
    model = create_model(n_input,n_out,train_x,train_y,test_x,test_y,rate)

    train_pred, test_pred = model.predict(train_x),model.predict(test_x)

    train_rmse_error +=  rmse_error(train_pred,train_y)
    test_rmse_error +=  rmse_error(test_pred,test_y)

  train_rmse_error, test_rmse_error = train_rmse_error/estimators, test_rmse_error/estimators
  rmse.append([rate,train_rmse_error,test_rmse_error])

In [ ]:
# predicted values
predicted = model.predict(Inputs_plot)
actual = Targets_plot

In [ ]:
# Next 1 day plotting
comp_df = pd.DataFrame({"actual":actual[0].flatten(),"predicted":predicted[0].flatten()})
sns.lineplot(data = comp_df, legend="full")

In [ ]:
# Next 2 day plotting
comp_df = pd.DataFrame({"actual":actual[0:2].flatten(),"predicted":predicted[0:2].flatten()})
sns.lineplot(data = comp_df)

In [ ]:
# Next 7 day plotting
comp_df = pd.DataFrame({"actual":actual[0:7].flatten(),"predicted":predicted[0:7].flatten()})
sns.lineplot(data = comp_df)

In [ ]:
train_pred, test_pred = model.predict(train_x),model.predict(test_x)

In [ ]:
# Performance matrix on training set
{'explained_variance_score':r2_score(train_pred,train_y),
      'mean_squared_error':mean_squared_error(train_pred,train_y),
          'mean_absolute_error':mean_absolute_error(train_pred,train_y)
}


In [ ]:
#Performance matrix on test set
{'explained_variance_score':r2_score(test_pred,test_y),
    'mean_squared_error':mean_squared_error(test_pred,test_y),
          'mean_absolute_error':mean_absolute_error(test_pred,test_y)}
